In [5]:
import pygame
import random
import math

pygame.init()

# Constants
WINDOW_SIZE = (600, 600)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
FONT_COLOR = (0, 0, 0)
FONT_SIZE = 24
step_size = 3
tolerance = 10
ARROW_LENGTH = 20  # Length of direction arrows

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("2D Environment with Moving Target")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

# Agent positions and goal flags
agent1_pos = [300, 300]  # W (green)
agent2_pos = [300, 300]  # H (blue, user-controlled)
agent3_pos = [300, 300]  # X (red, weighted average)
# Add direction vectors for each agent
agent1_dir = [0, 0]
agent2_dir = [0, 0]
agent3_dir = [0, 0]
reached_goal = False

# Target (goal) is placed randomly in the 2D space
target_pos = [random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])]

def distance(pos1, pos2):
    return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

def calculate_gamma():
    max_distance = math.sqrt(WINDOW_SIZE[0]**2 + WINDOW_SIZE[1]**2)
    distance_w = distance(agent1_pos, target_pos)
    distance_h = distance(agent2_pos, target_pos)
    
    # Normalize distances
    normalized_distance_w = distance_w / max_distance
    normalized_distance_h = distance_h / max_distance
    
    # Calculate gamma based on relative distances
    if normalized_distance_w + normalized_distance_h == 0:
        return 0.5
    else:
        gamma = normalized_distance_h / (normalized_distance_w + normalized_distance_h)
        
    gamma = gamma ** 2
    return gamma

def normalize_vector(dx, dy):
    length = math.sqrt(dx**2 + dy**2)
    if length == 0:
        return 0, 0
    return dx/length, dy/length

def draw_arrow(surface, color, start_pos, direction, length=ARROW_LENGTH):
    if direction[0] == 0 and direction[1] == 0:
        return
        
    # Calculate end point
    end_x = start_pos[0] + direction[0] * length
    end_y = start_pos[1] + direction[1] * length
    
    # Draw main line
    pygame.draw.line(surface, color, start_pos, (end_x, end_y), 2)
    
    # Calculate arrow head
    arrow_size = 7
    angle = math.atan2(direction[1], direction[0])
    arrow1_x = end_x - arrow_size * math.cos(angle + math.pi/6)
    arrow1_y = end_y - arrow_size * math.sin(angle + math.pi/6)
    arrow2_x = end_x - arrow_size * math.cos(angle - math.pi/6)
    arrow2_y = end_y - arrow_size * math.sin(angle - math.pi/6)
    
    # Draw arrow head
    pygame.draw.line(surface, color, (end_x, end_y), (arrow1_x, arrow1_y), 2)
    pygame.draw.line(surface, color, (end_x, end_y), (arrow2_x, arrow2_y), 2)

def move_agent1():
    global agent1_pos, reached_goal, agent1_dir
    dx = target_pos[0] - agent1_pos[0]
    dy = target_pos[1] - agent1_pos[1]
    distance_to_target = math.sqrt(dx**2 + dy**2)
    
    # Update direction vector
    agent1_dir = normalize_vector(dx, dy)
    
    if distance_to_target > step_size:
        move_ratio = step_size / distance_to_target
        agent1_pos[0] += dx * move_ratio
        agent1_pos[1] += dy * move_ratio
    else:
        agent1_pos[0] = target_pos[0]
        agent1_pos[1] = target_pos[1]
        reached_goal = True

def move_agent2(dx, dy):
    global agent2_pos, agent2_dir
    new_x = agent2_pos[0] + dx
    new_y = agent2_pos[1] + dy
    agent2_pos[0] = max(0, min(WINDOW_SIZE[0], new_x))
    agent2_pos[1] = max(0, min(WINDOW_SIZE[1], new_y))
    
    # Update direction vector
    if dx != 0 or dy != 0:
        agent2_dir = normalize_vector(dx, dy)

def update_agent3():
    global agent3_pos, agent3_dir
    gamma = calculate_gamma()
    new_x = gamma * agent1_pos[0] + (1 - gamma) * agent2_pos[0]
    new_y = gamma * agent1_pos[1] + (1 - gamma) * agent2_pos[1]
    
    # Update direction vector
    dx = new_x - agent3_pos[0]
    dy = new_y - agent3_pos[1]
    agent3_dir = normalize_vector(dx, dy)
    
    agent3_pos[0] = new_x
    agent3_pos[1] = new_y
    return gamma

def reset():
    global agent1_pos, agent2_pos, agent3_pos, reached_goal, target_pos, agent1_dir, agent2_dir, agent3_dir
    agent1_pos = [300, 300]
    agent2_pos = [300, 300]
    agent3_pos = [300, 300]
    agent1_dir = [0, 0]
    agent2_dir = [0, 0]
    agent3_dir = [0, 0]
    reached_goal = False
    target_pos = [random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])]

def render(gamma):
    screen.fill(WHITE)
    
    # Draw agents and their direction arrows
    pygame.draw.circle(screen, GREEN, (int(agent1_pos[0]), int(agent1_pos[1])), 10)
    pygame.draw.circle(screen, BLUE, (int(agent2_pos[0]), int(agent2_pos[1])), 10)
    pygame.draw.circle(screen, RED, (int(agent3_pos[0]), int(agent3_pos[1])), 10)
    pygame.draw.circle(screen, YELLOW, (int(target_pos[0]), int(target_pos[1])), 10)
    
    # Draw direction arrows
    draw_arrow(screen, GREEN, (int(agent1_pos[0]), int(agent1_pos[1])), agent1_dir)
    draw_arrow(screen, BLUE, (int(agent2_pos[0]), int(agent2_pos[1])), agent2_dir)
    draw_arrow(screen, RED, (int(agent3_pos[0]), int(agent3_pos[1])), agent3_dir)

    # Draw labels
    screen.blit(font.render("W", True, BLACK), (agent1_pos[0] - 5, agent1_pos[1] - 12))
    screen.blit(font.render("H", True, BLACK), (agent2_pos[0] - 5, agent2_pos[1] - 12))
    screen.blit(font.render("X", True, BLACK), (agent3_pos[0] - 5, agent3_pos[1] - 12))

    # Display gamma and formula
    gamma_text = font.render(f"Gamma: {gamma:.2f}", True, FONT_COLOR)
    screen.blit(gamma_text, (10, 10))
    formula_text = font.render(f"X = {gamma:.2f}W + {1-gamma:.2f}H", True, FONT_COLOR)
    screen.blit(formula_text, (10, 40))

    if reached_goal:
        reset_text = font.render("Goal Reached! Press R to reset", True, FONT_COLOR)
        screen.blit(reset_text, (150, 70))

    pygame.display.update()

# Main
running = True
clock = pygame.time.Clock()
gamma = 0.5  # Initialize gamma

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                reset()

    if not reached_goal:
        keys = pygame.key.get_pressed()
        dx, dy = 0, 0
        if keys[pygame.K_LEFT]:
            dx -= step_size
        if keys[pygame.K_RIGHT]:
            dx += step_size
        if keys[pygame.K_UP]:
            dy -= step_size
        if keys[pygame.K_DOWN]:
            dy += step_size

        if dx != 0 or dy != 0:
            move_agent2(dx, dy)
            move_agent1()
            gamma = update_agent3()

    render(gamma)
    clock.tick(30)

pygame.quit()